In [1]:
import SparkerMethods
SparkerMethods.runSpark(setMaster = False)

In [2]:
def objectiveLogs(inputPaths, keyword):
    import SearchExtractor, FinalizedRunners
    logs = FinalizedRunners.getPreparedLogsFromHDFS(inputPaths, filtering = False)
    return SearchExtractor.searchNProductLogsForSingleKeyword(logs, keyword)

In [29]:
import paths
keyword = 'iphone 7'
keyword = 'besiktas'
keyword_name =  keyword.replace(' ', '_')
inputPath = paths.joinPath(paths.joinPath(paths.HDFSRootFolder, 'weekAugust'), keyword_name + '/' + keyword_name + '_extractedLogs')
searchNProductLogs = objectiveLogs(inputPath, keyword)

hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/weekAugust/besiktas/besiktas_extractedLogs has been read by 2017-09-01 02:13:57.980268
1919 have been prepared by 2017-09-01 02:13:58.212044
1264 searches have found for besiktas by 2017-09-01 02:13:58.395393
3591 products have listed on searches for besiktas by 2017-09-01 02:13:59.094801
348 views have found for besiktas by 2017-09-01 02:13:59.269425
183 products have clicked on searches for besiktas by 2017-09-01 02:14:00.348077
331 cart and payments have found for besiktas by 2017-09-01 02:14:01.474924


In [30]:
import Sessionizer, NewProductPreferrer
(searches, viewedProductLogs, cartedOrPaidProductLogs) = searchNProductLogs
searches = searches.map(Sessionizer.idSetter)
productLogs = viewedProductLogs.union(cartedOrPaidProductLogs).map(Sessionizer.idSetter)
searchesNProducts = NewProductPreferrer.pairSearchesNProducts(searches, productLogs)
print(searchesNProducts.count())

6887


In [31]:
pairs1, pairedSearchesNProducts = NewProductPreferrer.getLabeledPairsOnSinglePage(searchesNProducts)
print(pairs1.count(), pairedSearchesNProducts.count())

45346 267


In [34]:
import LumberjackConstants as L
nextSearches = pairedSearchesNProducts.filter(lambda sp: sp[1][0][L.KEY_PAGENUM] > 1)
print(nextSearches.count())

121


In [44]:
#print(nextSearches.first()[1])

In [95]:
import PythonVersionHandler, SparkLogFileHandler, LumberjackConstants as L
searchesNPrevious = SparkLogFileHandler.sc_().parallelize([])
for id_key in [L.KEY_PERSISTENT_COOKIE,L.KEY_USER_ID_FROM_COOKIE,L.KEY_USER_ID,L.KEY_SESSION_ID]:
    subSearches = nextSearches.map(lambda kv: (kv[1][0][id_key], kv))
    searches1 = searches.map(lambda search: (search[id_key], search))
    subSearches = subSearches.join(searches1)
    searchesNPrevious = searchesNPrevious.union(subSearches)
searchesNPrevious = searchesNPrevious.map(lambda sp: sp[1])

In [99]:
print(searchesNPrevious.count())
searchesNPrevious = searchesNPrevious.map(lambda sp: str(sp)).distinct().map(lambda sp: eval(sp))
print(searchesNPrevious.count())

3018


In [109]:
searchesNPrevious1 = searchesNPrevious.filter(lambda sp: sp[0][1][0][L.KEY_TIMESTAMP] > sp[1][L.KEY_TIMESTAMP]\
     and sp[0][1][0][L.KEY_PAGENUM] > sp[1][L.KEY_PAGENUM] and not NewProductPreferrer.isProductIdOnSearch(sp[0][1][1][1], sp[1]))
#.filter(lambda sp: ))

In [110]:
print(searchesNPrevious1.count())
#print(searchesNPrevious.first()[1])

1231


In [114]:
searchesNPrevious2 = searchesNPrevious1.map(lambda sp: (sp[1], sp[0][1][1]))
pairs = searchesNPrevious2.flatMap(instanceListFromPrevious)
print(pairs.count())
pairs1 = pairs1.flatMap(NewProductPreferrer.labelPairs)
print(pairs1.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 5401.0 failed 1 times, most recent failure: Lost task 10.0 in stage 5401.0 (TID 351497, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/pyspark/rdd.py", line 2407, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2407, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2407, in pipeline_func
    return func(split, prev_func(split, iterator))
  Fil

In [111]:
def instanceListFromActions(sp, productOnPage = True):
    import LumberjackConstants as L
    if sp[1][1][L.KEY_MODULE] == L.KEY_MODULE_CART:
        coefficient = KEY_CART_COEFFICIENT
    elif sp[1][1][L.KEY_MODULE] == L.KEY_MODULE_PAYMENT:
        coefficient = KEY_PAYMENT_COEFFICIENT
    else:
        coefficient = KEY_PRODUCT_COEFFICIENT
    if isinstance(sp[1][1][L.KEY_ID], int):
        l = sp[0][L.KEY_ID_LIST][:sp[0][L.KEY_ID_LIST].index(sp[1][1][L.KEY_ID])] if productOnPage else sp[0][L.KEY_ID_LIST]
        return coefficient * [(sp[1][1][L.KEY_ID], l)]
    else:
        processedIds = [int(i) for i in sp[1][1][L.KEY_ID].split('%7C')]
        s = []
        for pid in processedIds:
            if pid in sp[0][L.KEY_ID_LIST]:
                l = sp[0][L.KEY_ID_LIST][:sp[0][L.KEY_ID_LIST].index(pid)] if productOnPage else sp[0][L.KEY_ID_LIST]
                s.extend(coefficient * [(pid, l)])
        return s

def instanceListFromPrevious(sp):
    return instanceListFromActions(sp, productOnPage = False)

In [35]:
print('Total:', pairedSearchesNProducts.count(), pairedSearchesNProducts.filter(lambda sp: sp[1][1][1][L.KEY_MODULE] != L.KEY_MODULE_ITEM).count())

Total: 267 38
